In [1]:
import sys
import pandas as pd
sys.path.append('..')
import os
import rbf_functions

In [2]:
rbfs = [rbf_functions.original_rbf,
        rbf_functions.squared_exponential_rbf,
        rbf_functions.inverse_quadratic_rbf,
        rbf_functions.inverse_multiquadric_rbf,
        rbf_functions.exponential_rbf,
        rbf_functions.matern32_rbf,
        rbf_functions.matern52_rbf,
       ]

# Load data

In [52]:
hv_global = pd.read_csv('./calculated_metrics/hv_refset_global.csv').iloc[0, -1]
averages = {}
for entry in rbfs:
    rbf = entry.__name__
    
    # normalize hv against reference set so reference set is 1
    df_hv = pd.read_csv(f"calculated_metrics/hv_{rbf}_global.csv")
    df_hv.hypervolume /= hv_global
    
    df_ei = pd.read_csv(f"calculated_metrics/ei_{rbf}_global.csv")
    df_gd = pd.read_csv(f"calculated_metrics/gd_{rbf}_global.csv")      

    avg_hv = df_hv.groupby('seed').tail(1).loc[:, 'hypervolume'].mean()
    avg_ei = df_ei.groupby('ei').tail(1).loc[:, 'ei'].mean()
    avg_gd = df_gd.groupby('gd').tail(1).loc[:, 'gd'].mean()
    averages[rbf] = dict(hypervolume=avg_hv, epsilon_indicator=avg_ei, 
                         generational_distance=avg_gd)


In [53]:
scores = pd.DataFrame.from_dict(averages).T
scores

,hypervolume,epsilon_indicator,generational_distance
original_rbf,0.716880,0.349520,0.022556
squared_exponential_rbf,0.641916,0.411422,0.020800
inverse_quadratic_rbf,0.333763,0.472414,0.030182
inverse_multiquadric_rbf,0.350160,0.582708,0.023650
exponential_rbf,0.151511,0.363626,0.042522
matern32_rbf,0.374623,0.356777,0.032786
matern52_rbf,0.334237,0.369322,0.031683


In [71]:
import os

paretosets = {}
size = {}
for entry in rbfs:
    name = entry.__name__
    # load variables
    varlist = []
    variables = []

    output_dir = os.path.abspath('./refsets/')
    for filename in os.listdir(output_dir):
        if filename == f'{name}_refset.csv':
            varlist.append(filename[:-4])
            df_vars = pd.read_csv(os.path.join(output_dir, filename))
            paretosets[name] = df_vars
            size[name] = len(df_vars)
    variables = df_vars.values
#     print(f"Loaded: {', '.join(varlist)}")

size

{'original_rbf': 2675,
 'squared_exponential_rbf': 2142,
 'inverse_quadratic_rbf': 1684,
 'inverse_multiquadric_rbf': 1636,
 'exponential_rbf': 1585,
 'matern32_rbf': 2268,
 'matern52_rbf': 1774}

In [72]:
scores['nr. of solutions'] = pd.Series(size)

In [73]:
scores

,hypervolume,epsilon_indicator,generational_distance,nr. of solutions
original_rbf,0.716880,0.349520,0.022556,2675
squared_exponential_rbf,0.641916,0.411422,0.020800,2142
inverse_quadratic_rbf,0.333763,0.472414,0.030182,1684
inverse_multiquadric_rbf,0.350160,0.582708,0.023650,1636
exponential_rbf,0.151511,0.363626,0.042522,1585
matern32_rbf,0.374623,0.356777,0.032786,2268
matern52_rbf,0.334237,0.369322,0.031683,1774


# set contributions

In [74]:
df_global = pd.read_csv(f"./refsets/global_refset.csv")

In [75]:
l_global = df_global.values.tolist()
contribution = {}
templ = []
for rbf in paretosets:
    x=0
    for solution in paretosets[rbf].values.tolist():
        if solution in l_global:
            templ.append(solution)
            x +=1
    contribution[rbf] = round(x/len(df_global), 3)
    rbfd[rbf] = pd.DataFrame(templ, columns=data.columns)
    templ = []
contribution

{'original_rbf': 0.743,
 'squared_exponential_rbf': 0.183,
 'inverse_quadratic_rbf': 0.02,
 'inverse_multiquadric_rbf': 0.052,
 'exponential_rbf': 0.0,
 'matern32_rbf': 0.001,
 'matern52_rbf': 0.002}

In [76]:
scores["set contribution"] = pd.Series(contribution)

In [77]:
scores

,hypervolume,epsilon_indicator,generational_distance,nr. of solutions,set contribution
original_rbf,0.716880,0.349520,0.022556,2675,0.743
squared_exponential_rbf,0.641916,0.411422,0.020800,2142,0.183
inverse_quadratic_rbf,0.333763,0.472414,0.030182,1684,0.020
inverse_multiquadric_rbf,0.350160,0.582708,0.023650,1636,0.052
exponential_rbf,0.151511,0.363626,0.042522,1585,0.000
matern32_rbf,0.374623,0.356777,0.032786,2268,0.001
matern52_rbf,0.334237,0.369322,0.031683,1774,0.002


In [78]:
scores.to_latex()

/var/folders/_t/l6cr0r0d1m108xdrmrtz7stsb_csl0/T/ipykernel_65689/2965949692.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  scores.to_latex()


'\\begin{tabular}{lrrrrr}\n\\toprule\n{} &  hypervolume &  epsilon\\_indicator &  generational\\_distance &  nr. of solutions &  set contribution \\\\\n\\midrule\noriginal\\_rbf             &     0.716880 &           0.349520 &               0.022556 &              2675 &             0.743 \\\\\nsquared\\_exponential\\_rbf  &     0.641916 &           0.411422 &               0.020800 &              2142 &             0.183 \\\\\ninverse\\_quadratic\\_rbf    &     0.333763 &           0.472414 &               0.030182 &              1684 &             0.020 \\\\\ninverse\\_multiquadric\\_rbf &     0.350160 &           0.582708 &               0.023650 &              1636 &             0.052 \\\\\nexponential\\_rbf          &     0.151511 &           0.363626 &               0.042522 &              1585 &             0.000 \\\\\nmatern32\\_rbf             &     0.374623 &           0.356777 &               0.032786 &              2268 &             0.001 \\\\\nmatern52\\_rbf             

In [79]:
scores.cols

array(['hypervolume', 'epsilon_indicator', 'generational_distance',
       'nr. of solutions', 'set contribution'], dtype=object)

In [87]:
scores[[ 'nr. of solutions', 'generational_distance',
        'epsilon_indicator', 'hypervolume', 'set contribution']].to_latex()

/var/folders/_t/l6cr0r0d1m108xdrmrtz7stsb_csl0/T/ipykernel_65689/1778145395.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  scores[[ 'nr. of solutions', 'generational_distance',


'\\begin{tabular}{lrrrrr}\n\\toprule\n{} &  nr. of solutions &  generational\\_distance &  epsilon\\_indicator &  hypervolume &  set contribution \\\\\n\\midrule\noriginal\\_rbf             &              2675 &               0.022556 &           0.349520 &     0.716880 &             0.743 \\\\\nsquared\\_exponential\\_rbf  &              2142 &               0.020800 &           0.411422 &     0.641916 &             0.183 \\\\\ninverse\\_quadratic\\_rbf    &              1684 &               0.030182 &           0.472414 &     0.333763 &             0.020 \\\\\ninverse\\_multiquadric\\_rbf &              1636 &               0.023650 &           0.582708 &     0.350160 &             0.052 \\\\\nexponential\\_rbf          &              1585 &               0.042522 &           0.363626 &     0.151511 &             0.000 \\\\\nmatern32\\_rbf             &              2268 &               0.032786 &           0.356777 &     0.374623 &             0.001 \\\\\nmatern52\\_rbf             